Import packages

In [1]:
import numpy as np
import pandas as pd
import datetime

Read all files 

In [2]:
# set the folder path
folder_path = "/Users/areguly6/Library/CloudStorage/OneDrive-CorvinusUniversityofBudapest/Research/XBRL/data/SEC_raw/2023q4/"

In [3]:
# Read the num dataset
num_path = "num.txt"
num_df = pd.read_csv(folder_path + num_path, delimiter="\t", header=0)

# Read the tag dataset
tag_path = "tag.txt"
tag_df = pd.read_csv(folder_path + tag_path, delimiter="\t", header=0)

# Read the sub dataset
sub_path = "sub.txt"
sub_df = pd.read_csv(folder_path + sub_path, delimiter="\t", header=0)

# Read the pre dataset
pre_path = "pre.txt"
pre_df = pd.read_csv(folder_path + pre_path, delimiter="\t", header=0)



Format data

In [4]:
print(sub_df.columns.values)
print(tag_df.columns.values)
print(num_df.columns.values)
print(pre_df.columns.values)

['adsh' 'cik' 'name' 'sic' 'countryba' 'stprba' 'cityba' 'zipba' 'bas1'
 'bas2' 'baph' 'countryma' 'stprma' 'cityma' 'zipma' 'mas1' 'mas2'
 'countryinc' 'stprinc' 'ein' 'former' 'changed' 'afs' 'wksi' 'fye' 'form'
 'period' 'fy' 'fp' 'filed' 'accepted' 'prevrpt' 'detail' 'instance'
 'nciks' 'aciks']
['tag' 'version' 'custom' 'abstract' 'datatype' 'iord' 'crdr' 'tlabel'
 'doc']
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']
['adsh' 'report' 'line' 'stmt' 'inpth' 'rfile' 'tag' 'version' 'plabel'
 'negating']


In [5]:
# assign correct datatypes 

# SUB
sub_df['cik'] = sub_df['cik'].astype(int)
sub_df['sic'] = sub_df['sic'].astype('Int64')
sub_df['ein'] = sub_df['ein'].astype(float) 
sub_df['wksi'] = sub_df['wksi'].astype('bool') 
sub_df['period'] = pd.to_datetime(sub_df['period'])
sub_df['fy'] = sub_df['fy'].astype('Int64') 
sub_df['filed'] = pd.to_datetime(sub_df['filed'])
sub_df['accepted'] = pd.to_datetime(sub_df['accepted'])
sub_df['prevrpt'] = sub_df['prevrpt'].astype('bool') 
sub_df['detail'] = sub_df['detail'].astype('bool') 
sub_df['nciks'] = sub_df['nciks'].astype('Int64')

# TAG
tag_df['custom'] = tag_df['custom'].astype('bool')
tag_df['abstract'] = tag_df['abstract'].astype('bool')

# Num
num_df['ddate'] = pd.to_datetime(num_df['ddate'])
# num_df['coreg'] = num_df['coreg'].astype(float)
num_df['qtrs'] = num_df['qtrs'].astype('Int64')
num_df['value'] = num_df['value'].astype(float)

# Pre
pre_df['report'] = pre_df['report'].astype(float)
pre_df['line'] = pre_df['line'].astype(float)
pre_df['inpth'] = pre_df['inpth'].astype('bool')

Data dimensions for merging

(more here: https://www.sec.gov/files/aqfs.pdf)

Notes:

    - SUB: 'adsh' is the unique key. 20 char EDGAR Accession Number, with dashes in positions 11-14
    - TAG: set of all tags used in submissions (standard and custom)
        - unique identifier by: 'tag' (tag used by filer) and 'version; (if standard tag the taxonomy of origin, otherwise == adsh)
    - NUM: data set of all numeric XBRL facts. unique key must be generated by combination of the following list:
        - adsh - EDAR accession number
        - tag - tag used by the filer
        -version - if a standard tag, the taxonomy of origin, otherwise equal to adsh
        - ddate - period end date
        - qtrs - duration in number of quarters
        - uom - unit of measure
        - coreg - coregistrant of the parent company registrant (if applicable)
    - PRE -- text assigned by the filer to each line item
        - adsh
        - report - sequential number of report within the statements
        - line - sequential number of line within a report

In [8]:
# Match to NUM
df = pd.merge( num_df, sub_df, how = 'left', on = 'adsh' )
df = pd.merge( df, tag_df, how = 'left', on = ['tag','version'] )
df = pd.merge( df, pre_df, how = 'left', on = ['adsh','tag','version'] )

In [10]:
df.shape

(2639066, 51)

## Common Filters:

- Company's name is stored in 'name' that is corresponds to the name of legal entity as recorded in EDGAR as of the filing date
    - note that this may change over time!
    - you can use: https://www.sec.gov/edgar/searchedgar/companysearch to search for company/person by name, ticker symbol or CIK
    - you can find archives and text format CIK-company names here: https://www.sec.gov/Archives/edgar/cik-lookup-data.txt
- Get yearly reports: 'form' == '10-K' | form == '10-K/A' leads to have 10Ks:
    ``` 
    df = df.loc[
    (df['form'] == '10-K') |
    (df['form'] == '10-K/A') ]
    ```
- Reported accounting method - US-GAAP: 'version' == 'us-gaap'
    ```
    df = df.loc[ df['version'] == 'us-gaap' ] 
    ```
- Report is done by the company itself: 'coreg' == NA (or missing, use '.isna()' ) 
    ```
    df = df.loc[ df['coreg'].isna() ] 
    ```

## Useful variables:

- 'ticker' that stands for the stock-exchange ticker: 
    ```
    df.['ticker'] = df['instance'].astype(str).str.split('-', expand=True)[0].str.upper()
    ```
        - note however, it is 'noisy' meaning many cases missleading. 
        - To get a full picture, use instead: https://www.sec.gov/include/ticker.txt and merge with data
- sectors can be identified by 'sic' value.
    - Fama-French classifications -- 12 for rough 48 for detailled (e.g. banks), 30 is generally ok for analysis
        - 48 classification: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_48_ind_port.html
        - 12 classification: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_12_ind_port.html
        - 30 classification: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_30_ind_port.html

## Further data

- Yahho finance: https://finance.yahoo.com/
    - can download stock prices -> stock returns